In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import patsy as pt
import datetime


# Training Data
data = pd.read_csv('midterm_sales_and_date_df.csv')

# order_purchase_timestamp as date
format = '%Y-%m-%d'
data['order_purchase_timestamp'] = pd.to_datetime(data['order_purchase_timestamp'], format=format)

# add variables using `order_purchase_timestamp`
data['year'] = data['order_purchase_timestamp'].dt.year
data['month'] = data['order_purchase_timestamp'].dt.month
data['weekday'] = data['order_purchase_timestamp'].dt.weekday
# 0 = Monday, 6 = Sunday
data['isBlackFriday'] = np.where((data['order_purchase_timestamp']=='2017-11-24') | (data['order_purchase_timestamp'] == '2018-11-27'), 1, 0)
data['isWeekend'] = np.where((data['weekday']== 5) | (data['weekday'] == 6), 1, 0)


# TEST SET
# Predict from 2018-09-01 to 2018-11-30
## make dataframe with this date range as column
test = pd.DataFrame(columns=['order_purchase_timestamp', 'sales_volume', 'year', 'month', 'weekday', 'isBlackFriday'])
test['order_purchase_timestamp'] = pd.date_range(start="2018-09-01",end="2018-11-30")
test['year'] = test['order_purchase_timestamp'].dt.year
test['month'] = test['order_purchase_timestamp'].dt.month
test['weekday'] = test['order_purchase_timestamp'].dt.weekday
# set to 0 to fill column
test['sales_volume'] = 0
# 0 = Monday, 6 = Sunday
test['isBlackFriday'] = np.where((test['order_purchase_timestamp']=='2017-11-24') | (test['order_purchase_timestamp'] == '2018-11-27'), 1, 0)
test['isWeekend'] = np.where((test['weekday']== 5) | (test['weekday'] == 6), 1, 0)


# concatonate data2 and test
dat = pd.concat([data, test])


In [15]:
# dat.iloc[:481] is the cut off for our training set in the validation process
dat.iloc[481:573]

,order_purchase_timestamp,sales_volume,year,month,weekday,isBlackFriday,isWeekend
481,2018-05-01,304,2018,5,1,0,0
482,2018-05-02,405,2018,5,2,0,0
483,2018-05-03,436,2018,5,3,0,0
484,2018-05-04,372,2018,5,4,0,0
485,2018-05-05,250,2018,5,5,0,1
...,...,...,...,...,...,...,...
568,2018-07-27,253,2018,7,4,0,0
569,2018-07-28,270,2018,7,5,0,1
570,2018-07-29,247,2018,7,6,0,1
571,2018-07-30,350,2018,7,0,0,0


In [9]:
# FOR VALIDATION
# predicting sales (total_price)
y, x = pt.dmatrices("sales_volume ~ C(year) + C(month) + C(weekday) + isWeekend + isBlackFriday", data = dat.iloc[:481], return_type='dataframe')


In [10]:
# Specify regression
reg = sm.OLS(endog=y, exog=x) 


# Fit regression with robust standard errors
fit = reg.fit().get_robustcov_results(cov_type='cluster', groups=dat.iloc[:481, dat.columns.get_loc('order_purchase_timestamp')])

# Print results
print(fit.summary())

                            OLS Regression Results                            
Dep. Variable:           sales_volume   R-squared:                       0.788
Model:                            OLS   Adj. R-squared:                  0.780
Method:                 Least Squares   F-statistic:                 3.259e+04
Date:                Fri, 16 Oct 2020   Prob (F-statistic):               0.00
Time:                        15:44:04   Log-Likelihood:                -2652.5
No. Observations:                 481   AIC:                             5345.
Df Residuals:                     461   BIC:                             5429.
Df Model:                          19                                         
Covariance Type:              cluster                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          95.4807     12.407     

/opt/conda/lib/python3.7/site-packages/statsmodels/base/model.py:1752: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 20, but rank is 19
  'rank is %d' % (J, J_), ValueWarning)


In [16]:
# VALIDATE
# here the data is the validation data - dat.iloc[481:573] from May-July 2018

# Store predictions and truth
xPred = pt.build_design_matrices([x.design_info], data.iloc[481:573])

pred = fit.predict(xPred).squeeze()
truth = data.iloc[481:573, 1]

# sales errors
errors = pred - truth

# Calculate Absolute Percentage Error
pce = np.abs(errors/truth)*100


# Print MSE, Mean Absolute Error, 
#   and Mean Abs Percentage Error
print("Mean Squared Error: %s" % 
	str(np.mean(errors**2)))
print("Mean Absolute Error: %s" % 
	str(np.mean(np.abs(errors))))
print("Mean Absolute Percentage Error: %s" 
	% str(np.mean(pce)))
## by what percentage of the actual value were we incorrect (56.% is pretty bad)
## may be because of variables included or omitted, or just difficult to predict

Mean Squared Error: 18563.000878016566
Mean Absolute Error: 115.46235258966634
Mean Absolute Percentage Error: 54.22020079581476
